<!-- # Student Dropout Prediction Model
This notebook contains data preprocessing, feature engineering, and machine learning model training for predicting student dropout. -->

In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_excel(r'student dropout.xlsx')
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")

Dataset shape: (649, 34)

Columns: ['School', 'Gender', 'Age', 'Address', 'Family_Size', 'Parental_Status', 'Mother_Education', 'Father_Education', 'Mother_Job', 'Father_Job', 'Reason_for_Choosing_School', 'Guardian', 'Travel_Time', 'Study_Time', 'Number_of_Failures', 'School_Support', 'Family_Support', 'Extra_Paid_Class', 'Extra_Curricular_Activities', 'Attended_Nursery', 'Wants_Higher_Education', 'Internet_Access', 'In_Relationship', 'Family_Relationship', 'Free_Time', 'Going_Out', 'Weekend_Alcohol_Consumption', 'Weekday_Alcohol_Consumption', 'Health_Status', 'Number_of_Absences', 'Grade_1', 'Grade_2', 'Final_Grade', 'Dropped_Out']


In [ ]:
print("Missing values:")
print(df.isnull().sum())
print("\nData types:")
print(df.dtypes)
print("\nTarget variable distribution:")
print(df['Dropped_Out'].value_counts())
print(df['Dropped_Out'].value_counts(normalize=True) * 100)

In [ ]:
# Separate features and target variable
X = df.drop('Dropped_Out', axis=1)
y = df['Dropped_Out']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()

print(f"Categorical columns: {categorical_cols}")
print(f"Numerical columns: {numerical_cols}")

In [ ]:
# Feature Engineering
X_processed = X.copy()

# Encode categorical variables
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X_processed[col] = le.fit_transform(X_processed[col])
    label_encoders[col] = le

# Create new features
# Average grade
X_processed['Average_Grade'] = (X_processed['Grade_1'] + X_processed['Grade_2'] + X_processed['Final_Grade']) / 3

# Grade improvement
X_processed['Grade_Improvement'] = X_processed['Final_Grade'] - X_processed['Grade_1']

# Family education level (average of mother and father education)
# X_processed['Family_Education_Avg'] = (X_processed['Mother_Education'] + X_processed['Father_Education']) / 2

# Study to free time ratio
X_processed['Study_Free_Ratio'] = X_processed['Study_Time'] / (X_processed['Free_Time'] + 1)  

X_processed['Total_Alcohol'] = X_processed['Weekend_Alcohol_Consumption'] + X_processed['Weekday_Alcohol_Consumption']

print(f"New features created. Dataset shape: {X_processed.shape}")
print(f"New columns: {X_processed.columns[-5:].tolist()}")

In [ ]:
# Feature correlation analysis
plt.figure(figsize=(12, 10))
correlation_matrix = X_processed.corr()
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Show correlation with target variable
target_corr = pd.concat([X_processed, y], axis=1).corr()['Dropped_Out'].sort_values(ascending=False)
print("\nCorrelation with target variable (Dropped_Out):")
print(target_corr)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Training set target distribution: {y_train.value_counts()}")
print(f"Test set target distribution: {y_test.value_counts()}")

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Train Logistic Regression Model
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_lr = lr_model.predict(X_test_scaled)
y_pred_proba_lr = lr_model.predict_proba(X_test_scaled)[:, 1]

# Calculate accuracy
lr_accuracy = accuracy_score(y_test, y_pred_lr)
print(f"Logistic Regression Accuracy: {lr_accuracy:.4f}")

# Classification report
print("\nLogistic Regression Classification Report:")
print(classification_report(y_test, y_pred_lr))

In [ ]:
# Train Random Forest Model for comparison
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]

rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")

print("\nRandom Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

In [ ]:
# Feature importance from Random Forest
feature_importance = pd.DataFrame({
    'feature': X_processed.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(data=feature_importance.head(5), x='importance', y='feature')
plt.title('Top 5 Feature Importances (Random Forest)')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

print("Top 10 Most Important Features:")
print(feature_importance.head(10))

In [ ]:
# Confusion Matrix Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Logistic Regression Confusion Matrix
cm_lr = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Logistic Regression\nConfusion Matrix')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Random Forest Confusion Matrix
cm_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens', ax=axes[1])
axes[1].set_title('Random Forest\nConfusion Matrix')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.show()

In [ ]:
# Model Comparison Summary
print(f"Logistic Regression Accuracy: {lr_accuracy:.4f}")
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")
print("\nBest performing model:", "Random Forest" if rf_accuracy > lr_accuracy else "Logistic Regression")
print(f"Accuracy difference: {abs(rf_accuracy - lr_accuracy):.4f}")

In [ ]:
# catboost model

from catboost import CatBoostClassifier
# from catboost_widget import CatBoostWidget

cat_model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, random_seed=42, verbose=100)
cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True, plot=True)

In [ ]:
# predictions
y_pred_cat = cat_model.predict(X_test)
y_pred_proba_cat = cat_model.predict_proba(X_test)[:, 1]

# test accuracy
cat_accuracy = accuracy_score(y_test, y_pred_cat)
print(f"CatBoost Accuracy: {cat_accuracy:.4f}")
print("\nCatBoost Classification Report:")
print(classification_report(y_test, y_pred_cat))


In [ ]:
# Feature importance from catboost
feature_importance_cat = pd.DataFrame({
    'feature': X_processed.columns,
    'importance': cat_model.feature_importances_
}).sort_values('importance', ascending=False)

# display feature importance
plt.figure(figsize=(10, 8))
sns.barplot(data=feature_importance_cat.head(10), x='importance', y='feature')
plt.title('Top 5 Feature Importances (CatBoost)')

In [ ]:
# confusion matrix visualization
cm_cat = confusion_matrix(y_test, y_pred_cat)
sns.heatmap(cm_cat, annot=True, fmt='d', cmap='Purples')
plt.title('CatBoost\nConfusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')

In [ ]:
# # Save the trained models and preprocessors
# import joblib

# # Save models
# joblib.dump(lr_model, 'logistic_regression_model.pkl')
# joblib.dump(rf_model, 'random_forest_model.pkl')
# joblib.dump(scaler, 'feature_scaler.pkl')
# joblib.dump(label_encoders, 'label_encoders.pkl')
# joblib.dump(cat_model, 'catboost_model.pkl')

# print("Models and preprocessors saved successfully!")
# print("Files saved:")
# print("- logistic_regression_model.pkl")
# print("- random_forest_model.pkl")
# print("- feature_scaler.pkl")
# print("- label_encoders.pkl")
# print("- catboost_model.pkl")